In [1]:
import xarray as xr
import numpy as np
import os
import matplotlib.pyplot as plt
import os, sys
sys.path.append(os.path.abspath('../TEF'))
import TEF_Variables as tef

In [2]:
# Calculate TEF transports
def tef_transport(datapath, case_id, xi):    
    
    State0 = xr.open_dataset(datapath+'/state_' + str(format(case_id,'03d')) + '.nc')
    Grid = xr.open_dataset(datapath+'/grid_' + str(format(case_id,'03d')) + '.nc')
    State = State0.isel(T=~State0.get_index("T").duplicated())
    # Confine to the range of fjord
    state = State.isel(X=range(260), Xp1=range(261), Y=range(35,45), T=range(1,len(State.T)))
    grid = Grid.isel(X=range(260), Xp1=range(261), Y=range(35,45))
    
    s = state.S.data.mean(2) # Salinity in T,Z,X
    u = (state.U.data[:,:,:,1:].mean(2) + state.U.data[:,:,:,:-1].mean(2)) / 2 # Along channel velocity
    ot = state.T.data # Time in seconds


    HFacC1 = grid.HFacC.data.mean(1)
    dyF1 = grid.dyF.data.mean(0)
    drF1 = grid.drF.data
    gridA = np.broadcast_to(drF1[:, np.newaxis], HFacC1.shape) * np.broadcast_to(dyF1[np.newaxis, :], HFacC1.shape) * HFacC1 # Grid Area
    da = np.broadcast_to(gridA[np.newaxis,:,:], u.shape)

    S = state.S.data
    U = (state.U.data[:,:,:,1:] + state.U.data[:,:,:,:-1]) / 2
    drF = np.broadcast_to(grid.drF.data[np.newaxis, :, np.newaxis, np.newaxis], U.shape)
    dyF = np.broadcast_to(grid.dyF.data[np.newaxis, np.newaxis, :, :], U.shape)
    HFacC = np.broadcast_to(grid.HFacC.data[np.newaxis, :, :, :], U.shape)
    DA = drF * dyF * HFacC
    
    
    Qin = np.empty(len(xi))
    Qout = np.empty(len(xi))
    Sin = np.empty(len(xi))
    Sout = np.empty(len(xi))
    
    t0 = 89 # 274
    # Exclude the upper (surface) layer for TEF calculation
    for t in range(U.shape[0]):
        
        Uzx = u[t,:,:] # Steady-state along-channel velocity
        for j in range(Uzx.shape[1]):
            if any(Uzx[:,j]>0)==False:
                pass
            else:
                mid = np.where(Uzx[:,j]>0)[0][0]
                if mid>0:
                    S[t,:mid,:,j] = 0
                    U[t,:mid,:,j] = 0
                else:
                    pass
    
    
    for i in range(len(xi)):
               
        tef_q1, tef_vel1, tef_da1, tef_qs1, tef_qs21, sbins1 = tef.process_section(U,S,DA,ot,xi[i],23,testing=False)

        qin1, qout1, sin1, sout1 = tef.bulk_calc(tef_q1, tef_vel1, tef_da1, tef_qs1, tef_qs21, sbins1, ot)
    
        Qin[i] = qin1[t0:].mean() # Time averaging from 1+t0 hour
        Qout[i] = qout1[t0:].mean()
        Sin[i] = sin1[t0:].mean() # Time averaging from 1+t0 hour
        Sout[i] = sout1[t0:].mean()
        
    return Qin, Qout, Sin, Sout

In [ ]:
# Obtain reflux coefficients and entrainment flux Qe
def reflux(datapath, case_id, xi):

    State0 = xr.open_dataset(datapath+'/state_' + str(format(case_id,'03d')) + '.nc')
    Grid = xr.open_dataset(datapath+'/grid_' + str(format(case_id,'03d')) + '.nc')
    State = State0.isel(T=~State0.get_index("T").duplicated())
    # Confine to the range of fjord
    state = State.isel(X=range(260), Xp1=range(261), Y=range(35,45), T=range(1,len(State.T)))
    grid = Grid.isel(X=range(260), Xp1=range(261), Y=range(35,45))


    s = state.S.data.mean(2) # Salinity in T,Z,X
    u = (state.U.data[:,:,:,1:].mean(2) + state.U.data[:,:,:,:-1].mean(2)) / 2 # Along channel velocity
    ot = state.T.data # Time in seconds


    HFacC1 = grid.HFacC.data.mean(1)
    dyF1 = grid.dyF.data.mean(0)
    drF1 = grid.drF.data
    gridA = np.broadcast_to(drF1[:, np.newaxis], HFacC1.shape) * np.broadcast_to(dyF1[np.newaxis, :], HFacC1.shape) * HFacC1 # Grid Area
    da = np.broadcast_to(gridA[np.newaxis,:,:], u.shape)

    S = state.S.data
    U = (state.U.data[:,:,:,1:] + state.U.data[:,:,:,:-1]) / 2
    drF = np.broadcast_to(grid.drF.data[np.newaxis, :, np.newaxis, np.newaxis], U.shape)
    dyF = np.broadcast_to(grid.dyF.data[np.newaxis, np.newaxis, :, :], U.shape)
    HFacC = np.broadcast_to(grid.HFacC.data[np.newaxis, :, :, :], U.shape)
    DA = drF * dyF * HFacC
    
    
    Qin = np.empty(len(xi))
    Qout = np.empty(len(xi))
    Sin = np.empty(len(xi))
    Sout = np.empty(len(xi))
    t0 = 89 # 274
#    Exclude the upper (surface) layer for TEF calculation
#     for t in range(U.shape[0]):
        
#         Uzx = u[t,:,:] # Steady-state along-channel velocity
#         for j in range(Uzx.shape[1]):
#             if any(Uzx[:,j]>0)==False:
#                 pass
#             else:
#                 mid = np.where(Uzx[:,j]>0)[0][0]
#                 if mid>0:
#                     S[t,:mid,:,j] = 0
#                     U[t,:mid,:,j] = 0
#                 else:
#                     pass
    
    Uzx = u[t0:,:,:].mean(0) # Steady-state along-channel velocity
    for j in range(Uzx.shape[1]):
        if any(Uzx[:,j]>0)==False:
            pass
        else:
            mid = np.where(Uzx[:,j]>0)[0][0]
            if mid>0:
                S[:,:mid,:,j] = 0
                U[:,:mid,:,j] = 0
            else:
                pass
    
    for i in range(len(xi)):
    
        tef_q1, tef_vel1, tef_da1, tef_qs1, tef_qs21, sbins1 = tef.process_section(U,S,DA,ot,xi[i],23,testing=False)

        qin1, qout1, sin1, sout1 = tef.bulk_calc(tef_q1, tef_vel1, tef_da1, tef_qs1, tef_qs21, sbins1, ot)
    
        Qin[i] = qin1[t0:].mean() # Time averaging from 1+t0 hour
        Qout[i] = qout1[t0:].mean()
        Sin[i] = sin1[t0:].mean() # Time averaging from 1+t0 hour
        Sout[i] = sout1[t0:].mean()
        
    a002 = (Sin[0]/Sout[0])*(Sout[1]-Sout[0])/(Sout[1]-Sin[0])
    a001 = (Qin[0]-Qin[1]) / Qin[0]    
    a11 = (Sout[1]/Sin[1])*(Sin[1]-Sin[0])/(Sout[1]-Sin[0])    
    a01 = (Sout[1]/Sout[0])*(Sout[0]-Sin[0])/(Sout[1]-Sin[0])
    
    a10 = (Sin[0]/Sin[1])*(Sout[1]-Sin[1])/(Sout[1]-Sin[0])
    
    #a00 = -(Qout[0]/Qin[0])*(Sout[1]-Sout[0])/(Sout[1]-Sin[0])
    #a11 = -(Qin[1]/Qout[1])*(Sin[1]-Sin[0])/(Sout[1]-Sin[0])
    #a01 = (Qin[1]/Qin[0])*(Sout[1]-Sin[1])/(Sout[1]-Sin[0])
    #a10 = (Qout[0]/Qout[1])*(Sout[0]-Sin[0])/(Sout[1]-Sin[0])
    
    q0 = Qin[0]
    q1 = -Qout[-1]
    Q0 = -Qout[0]
    Q1 = Qin[-1]
    f0 = Sin[0]*q0
    f1 = Sout[-1]*q1
    F0 = Sout[0]*Q0
    F1 = Sin[-1]*Q1
    
    A = np.array([[q1, q0, 0, 0], [f1, f0, 0, 0], [0, 0, q1, q0], [0, 0, f1, f0]])
    B = np.array([Q1, F1, Q0, F0])
    X = np.linalg.solve(A,B) # a11, a01, a10, a00
    
    Qe = Q0 - q1
    
#     if (a11 < 0.1) or (a11 > 1):
#         a22 = a001
#     else:
#         a22 = a002
    if (X[-1] < 0) or (X[-1] > 1):
        a22 = a001
        a11 = 0
    else:
        a22 = X[-1]
        a11 = X[0]
    
    return a11,a22,q1,q0,Q1,Q0

In [3]:
def vol_temp(datapath, case_id, xi):
    

    State0 = xr.open_dataset(datapath+'/state_' + str(format(case_id,'03d')) + '.nc')
    Grid = xr.open_dataset(datapath+'/grid_' + str(format(case_id,'03d')) + '.nc')
    State = State0.isel(T=~State0.get_index("T").duplicated())    
    # Confine to the range of fjord
    state = State.isel(X=range(260), Xp1=range(261), Y=range(35,45), T=range(1,len(State.T)))
    grid = Grid.isel(X=range(260), Xp1=range(261), Y=range(35,45))

    U = (state.U.data[:,:,:,1:] + state.U.data[:,:,:,:-1]) / 2 # Along-channel velocity
    
    drF = np.broadcast_to(grid.drF.data[np.newaxis, :, np.newaxis, np.newaxis], U.shape)
    dyF = np.broadcast_to(grid.dyF.data[np.newaxis, np.newaxis, :, :], U.shape)
    HFacC = np.broadcast_to(grid.HFacC.data[np.newaxis, :, :, :], U.shape)
    DA = drF * dyF * HFacC
    
    rA = np.broadcast_to(grid.rA.data[np.newaxis, np.newaxis, :, :], U.shape)
    CV = rA * drF * HFacC

    
    t0 = 89 # 274
    da = DA[t0:,:,:,xi[0]+1:xi[1]+1].mean(axis=(0,2))
    cv = CV[t0:,:,:,xi[0]+1:xi[1]+1].mean(axis=(0,2))
    s = state.S.data[t0:,:,:,xi[0]+1:xi[1]+1].mean(axis=(0,2))
    temp = state.Temp.data[t0:,:,:,xi[0]+1:xi[1]+1].mean(axis=(0,2))
    u = U[t0:,:,:,xi[0]+1:xi[1]+1].mean(axis=(0,2))
    sma = np.ma.masked_where(s==0, s)
    topo = np.ma.getmask(sma) # Masked Topography
    uma = np.ma.MaskedArray(u, mask=topo)
    tma = np.ma.MaskedArray(temp, mask=topo)
    
    
    Tv1 = np.empty(uma.shape[1])
    Tv2 = np.empty(uma.shape[1])
    Vol1 = np.empty(uma.shape[1])
    Vol2 = np.empty(uma.shape[1])
    Tvol1 = np.empty(uma.shape[1])
    Tvol2 = np.empty(uma.shape[1])
    L = np.empty(uma.shape[1])
    
    for i in range(uma.shape[1]):

        # Exclude the surface layer 
        if any(uma[:,i]>0)==False:
            pass
        else:
            mid = np.where(uma[:,i]>0)[0][0]
            if mid>0:
                uma[:mid,i] = 0
            else:
                pass
            
        if any(uma[:,i]<0)==False:
            pass
            
        else:
            #meanS = np.abs(sma[:,i] - (sma[:,i].max() + sma[:,i].min()) / 2)
            #l = np.argmin(meanS)
            l = np.where(uma[:,i] < 0)[-1][0]    
            Tv1[i] = np.sum(tma[:l,i]*da[:l,i]*uma[:l,i]) / np.sum(da[:l,i]*uma[:l,i])
            Tv2[i] = np.sum(tma[l:,i]*da[l:,i]*uma[l:,i]) / np.sum(da[l:,i]*uma[l:,i])
            Vol1[i] = cv[:l,i].sum()
            Vol2[i] = cv[l:,i].sum()
            Tvol1[i] = np.sum(tma[:l,i]*cv[:l,i])
            Tvol2[i] = np.sum(tma[l:,i]*cv[l:,i])
            L[i] = l
            
        Tf = Tv1[:-1].mean()
        Ts = Tv2[:-1].mean()
        #Tf = Tvol1[:-1].sum()/Vol1[:-1].sum()
        #Ts = Tvol2[:-1].sum()/Vol2[:-1].sum()
        Ts_in = Tv2[-1]
        
    return Tf, Ts, Ts_in

In [4]:
# Obtain submarine melting Qsm
def IFA(datapath, case_id):
    # Grid areas
    Area = np.empty([90, 10])
    Area[:20,:] = 400
    Area[20:50,:] = 800
    Area[50:,:] = 1200
    file0 = xr.open_dataset(datapath+'/icefrntA_' + str(format(case_id,'03d')) + '.nc')
    file = file0.isel(T=~file0.get_index("T").duplicated()) 
    t0 = 89
    tn = len(file.T)
    state = file.isel(Y=range(35,45), T=range(t0,tn))
    MR = state.icefrntA.isel(X=1).data.mean(0) # Melt rate at the icefront
    Qsm = (MR*Area).sum()/(24*3600)
    return Qsm

In [ ]:
pathA = '/work/oceans/wbao/MITgcm_results/iceplume/Sal_Linear_minhs'
pathB = '/work/oceans/wbao/MITgcm_results/iceplume/Sal_Linear_nosill'
case = np.array([1, 2, 3, 4])

QsmA = np.empty(len(case))
QsmB = np.empty(len(case))


for j in range(len(case)):

    QsmA[j] = IFA(pathA,case[j])
    QsmB[j] = IFA(pathB,case[j])

QsmA, QsmB

In [5]:
path1 = '/work/oceans/wbao/MITgcm_results/iceplume/1_BaseCase'
path2 = '/work/oceans/wbao/MITgcm_results/iceplume/2_Qsg_maxhs'
path3 = '/work/oceans/wbao/MITgcm_results/iceplume/4_Sz_minhs'
path4 = '/work/oceans/wbao/MITgcm_results/iceplume/Tide_minhs'
case = np.array([1, 2, 3, 4, 5])
hsr = np.array([0.04,0.06,0.08,0.10,0.12])
Qsg = np.array([25, 50, 100, 250, 500, 1000])

xrange = np.array([120,235]) # X index range for the sill segment


Qin1 = np.empty(len(case))
Qin2 = np.empty(len(case))
Qout1 = np.empty(len(case))
Qout2 = np.empty(len(case))
Qin1_adj = np.empty(len(case))
Qin2_adj = np.empty(len(case))
Qout1_adj = np.empty(len(case))
Qout2_adj = np.empty(len(case))

Sin1 = np.empty(len(case))
Sin2 = np.empty(len(case))
Sout1 = np.empty(len(case))
Sout2 = np.empty(len(case))
alp11 = np.empty(len(case))
alp22 = np.empty(len(case))

Ts = np.empty(len(case))
Tf = np.empty(len(case))
Ts_in = np.empty(len(case))
Qsm = np.empty(len(case))

for j in range(len(case)):

    Qin, Qout, Sin, Sout = tef_transport(path4,case[j], xrange)
    
    Qin1[j] = Qin[0]
    Qin2[j] = -Qout[-1]
    Qout1[j] = -Qout[0]
    Qout2[j] = Qin[-1]
    Sin1[j] = Sin[0]
    Sin2[j] = Sout[-1]
    Sout1[j] = Sout[0]
    Sout2[j] = Sin[-1]
    
    alpha, Qin1_adj[j], Qin2_adj[j], Qout1_adj[j], Qout2_adj[j] = tef.efflux_reflux(Qin, Qout, Sin, Sout, error=False)
    
#     if (alpha[-1] < 0) or (alpha[-1] > 1):
#         alp11[j] = (Qin1_adj[j]-Qout2_adj[j]) / Qin1_adj[j]
#         alp22[j] = 0
#     else:
    alp11[j] = alpha[-1]
    alp22[j] = alpha[0]
    
    
    Tf[j], Ts[j], Ts_in[j] = vol_temp(path4,case[j],xrange)

    Qsm[j] = IFA(path4,case[j])


  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120


/home/1959/Scripts/Jupyter-Iceplume/TEF/TEF_Variables.py:269: RuntimeWarning: invalid value encountered in true_divide
  Sm = QSm/Qm
/home/1959/Scripts/Jupyter-Iceplume/TEF/TEF_Variables.py:273: RuntimeWarning: invalid value encountered in true_divide
  S2m = QS2m/Qm


  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120


In [13]:
#Tse = (Ts_in*Qin2*(1-alp22) + Tf*Qin1*alp11) / Qout1
#Tr = Tse / Ts_in
#Tse1 = (Ts_in*Qin2*(1-alp22) + Tf*Qin1*alp11) / Qout1
#Tse2 = (Ts_in*Qin2_adj*(1-alp22) + Tf*Qin1_adj*alp11) / Qout1_adj
#Tse1, Tse2, Ts
Qin1_adj-Qout2_adj, (Qin2_adj+Qout2_adj)/2, alp11, alp22, (Qin1_adj-Qout2_adj)/Qin1_adj, Ts, (Qin2_adj-Qout1_adj)/Qin2_adj

(array([6329.56148222, 6104.48178636, 5594.89984592, 4851.71897906,
        5228.62158002]),
 array([ 2552.09146001,  2784.28731547,  3887.85909901,  5375.52918527,
        14619.36073262]),
 array([0.714578  , 0.72686785, 0.76944392, 0.80404705,        nan]),
 array([0.04379473, 0.16954694, 0.48368894, 0.67170154,        nan]),
 array([0.70277548, 0.67694333, 0.58125379, 0.46698783, 0.21017625]),
 array([9.55162697e+00, 9.55089659e+00, 9.54243224e+00, 9.52820826e+00,
        1.22605432e+04]),
 array([-2.60773658, -2.29894387, -1.49394666, -0.9306271 , -0.54521467]))

In [ ]:
for i in range(len(alp11)):
    if (alp22[i] < 0) or (alp11[i] > 1):
        alp11[i] = (Qin1_adj[i]-Qout2_adj[i]) / Qin1_adj[i]
        alp22[i] = 0
    else:
        alp11[i] = alp11[i]
        alp22[i] = alp22[i]

In [ ]:
alp11[-1] = (Qin1_adj[-1]-Qout2_adj[-1]) / Qin1_adj[-1]

alp11, alp22

In [ ]:
alp22[-1] = 0
alp11, alp22

In [ ]:
# Save outputs
#U0 = np.array([1e-3, 2e-3, 3e-3, 4e-3, 5e-3])
#U0 = np.array([1e-4, 2e-4, 3e-4])

parameter_ds = xr.Dataset(
    data_vars={'S0z' : case,
    'alpha11' : alp11,
    'alpha22' : alp22,
    'Qin1' : Qin1_adj,
    'Qin2' : Qin2_adj,
    'Qout1' : Qout1_adj,
    'Qout2' : Qout2_adj,
    'Ts_in' : Ts_in,
    'Tf' : Tf,
    'Ts_model' : Ts})

# parameter_ds = xr.Dataset(
#     data_vars={'hsh' : hsr,
#     'alpha11' : alp11,
#     'alpha22' : alp22,
#     'Qin1' : Qin1,
#     'Qin2' : Qin2,
#     'Qout1' : Qout1,
#     'Qout2' : Qout2,
#     'Qin1_adj' : Qin1_adj,
#     'Qin2_adj' : Qin2_adj,
#     'Qout1_adj' : Qout1_adj,
#     'Qout2_adj' : Qout2_adj,
#     'Sin1' : Sin1,
#     'Sin2' : Sin2,
#     'Sout1' : Sout1,
#     'Sout2' : Sout2,
#     'Ts_in' : Ts_in,
#     'Tf' : Tf,
#     'Ts' : Ts,
#     'Qsm' : Qsm})

outdir = "/home/1959/Parameters/FjordModeling/Eq4/"
if not os.path.exists(outdir):
    os.makedirs(outdir)

parameter_ds.to_netcdf(outdir + '3-Strat.nc')


In [ ]:
path = '/work/oceans/wbao/MITgcm_results/iceplume/4_Sz_maxhs'
case = np.array([1, 2, 3, 4])
#hsr = np.array([0.04,0.06,0.08,0.10,0.12])
#Qsg = np.array([25,50,100,250,500,1000])
#T0 = np.array([2,4,6,8,12])
U0 = np.array([1e-4, 2e-4, 3e-4]) # Tidal amplitude at EBC


xrange = np.array([120,235]) # X index range for the sill segment

alp11 = np.empty(len(case))
alp22 = np.empty(len(case))
Qin0 = np.empty(len(case))
Qin1 = np.empty(len(case))
Qout0 = np.empty(len(case))
Qout1 = np.empty(len(case))
Ts = np.empty(len(case))
Tf = np.empty(len(case))
Ts_in = np.empty(len(case))



Qsm = np.empty(len(case))

for j in range(len(case)):
#     alp11_1[j], alp00_1[j], Qin1_1[j], Qin0_1[j], Qout1_1, Qout0_1[j] = reflux(qsg,hs[j],xrange1)
#     Tf_1[j], Ts_1[j], Ts_in_1[j] = vol_temp(qsg,hs[j],xrange1)
    
#     alp11_2[j], alp00_2[j], Qin1_2[j], Qin0_2[j], Qout1_2, Qout0_2[j] = reflux(qsg,hs[j],xrange2)
#     Tf_2[j], Ts_2[j], Ts_in_2[j] = vol_temp(qsg,hs[j],xrange2)
    alp11[j], alp22[j], Qin1[j], Qin0[j], Qout1[j], Qout0[j] = reflux(path,case[j],xrange)
    Tf[j], Ts[j], Ts_in[j] = vol_temp(path,case[j],xrange)

    Qsm[j] = IFA(path,case[j])

# Ts inside fjord
#Tse_1 = (Ts_in_1*Qin1_1*(1-alp11_1) + Tf_1*Qin0_1*alp00_1) / Qout0_1

# Ts at sill
#Tse_2 = (Ts_in_2*Qin1_2*(1-alp11_2) + Tf_2*Qin0_2*alp00_2) / Qout0_2

Tse = (Ts_in*Qin1*(1-alp11) + Tf*Qin0*alp22) / Qout0
Tr = Tse / Ts_in

Qe = Qout0 - Qin1
#Qe2 = Qout1 - Qin0

In [ ]:
Qsm

In [ ]:
# Save outputs
TEF_ds = xr.Dataset(
    data_vars={'hsr' : 0.12,
    'alpha11' : alp11,
    'alpha22' : alp22,
    'Qe' : Qe,
    'Ts_in' : Ts_in,
    'Ts_e' : Tse,
    'Ts' : Ts,
    'Qsm' : Qsm})

#TEF_ds = xr.Dataset(
#    data_vars={'Qsm' : Qsm})

#outdir = "/Volumes/Extreme SSD/MITgcm outputs/Qsg_comp/"
#outdir = "/Volumes/Extreme SSD/MITgcm outputs/Strat_comp/"
#outdir = "/Volumes/Extreme SSD/MITgcm outputs/Cd_comp/"
outdir = "/home/1959/Parameters/"
if not os.path.exists(outdir):
    os.makedirs(outdir)

TEF_ds.to_netcdf(outdir + 'Sz_maxhs.nc')
